<a href="https://colab.research.google.com/github/tknr/google_colab_stable_diffusion/blob/main/Stable_Diffusion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title セットアップ { display-mode: "form" }

# ライブラリのインストール
!pip install diffusers==0.2.4 transformers scipy ftfy googletrans==4.0.0-rc1

# アクセス・トークン設定
Access_Token="********" #@param {type:"string"}

# パイプライン構築
from diffusers import StableDiffusionPipeline
pipe = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4", use_auth_token=Access_Token)
pipe.to("cuda")

In [ ]:
#@title 画像生成 { display-mode: "form" }
import random
import torch
from googletrans import Translator

# 生成
prompt = "female japanese dj" #@param {type:"string"}
print(prompt)

translate_src = 'en' #@param ['ja','en']

if translate_src != 'en' :
  translator = Translator()
  translated = translator.translate(prompt, src=translate_src,dest='en')
  print(translated)
  print(translated.text)
  prompt = translated.text

out_pathes = []
num_images = 2 #@param {type:"slider", min:1, max:10, step:1}


prompts =[ prompt ] * num_images

width = 512 #@param [512,256,128]
height = 512 #@param [512,256,128] 
seed = 0 #@param {type:"slider", min:0, max:65535, step:1}
num_inference_steps=50 #@param {type:"slider", min:1, max:50, step:1}

if seed == 0:
  seed = random.randrange(1,65535)
  print(seed)

generator = torch.Generator("cuda").manual_seed(seed)

result = pipe(prompts,width=width,height=height,num_inference_steps=num_inference_steps,generator=generator)

print(result)

images = result["sample"]

for index,image in enumerate(images):

  # 保存
  sentence = prompt.replace(' ','_')
  out_path = "%s_%dx%d_seed%d_nis%d_%d.png" % (sentence,width,height,seed,num_inference_steps,index)
  print(out_path)
  image.save(out_path)
  out_pathes.append(out_path)

  # 表示
  from IPython.display import Image,display
  display(Image(out_path))

In [ ]:
#@title 画像のダウンロード(たぶんchrome only) { display-mode: "form" }
from google.colab import files
for index,out_path in enumerate(out_pathes):
  files.download(out_path)